# Case Study 1: Predicting Location via Indoor Positioning Systems (IPS)
## MSDS 7333: Quantifying the World
### Andy Nguyen, Michael Wolfe, Josepha Caguioa

## Objective
The floor plan for the building shows six access points. However, the data contain seven access points with roughly the expected number of signals (166 location 8 orientations 110 replications 146,080 measurements). Two of the MAC addresses seem to be for the same access point. Measurements from the MAC address 00:0f:a3:39:e1:c0 were kept while measurements to the 00:0f:a3:39:dd:cd address were eliminated. Conduct a more thorough data analysis into these two MAC addresses. Did we make the correct decision? Does swapping out the one we kept for the one we discarded improve the prediction?

### Guidelines

Using the OFFLINE data and **2 different clustering methods**, predict the location of the ONLINE dataset.

1. Describe how you prepared the data.
2. Describe how you estimated your error and found the best fit ASSUMING you CANNOT USE THE ONLINE DATA. Use tables and/or charts with analysis.
3. Describe the best fit for the data using tables and/or charts with analysis.
4. There are 2 macIDs located at the same position (the researchers used 00:0f:a3:39:e1:c0 and excluded 00:0f:a3:39:dd:cd for their analysis). Does one give better performance than the other?
    * Note: to effectively answer the below, you’d need to run (3) different scenarios (this aligns to question 9 from the text; page 41)
5. What about using them both? (compare the result of “c0 & cd” against “c0-only” and “cd-only”)
6. What are the drawback (if any of using this method to real-time locate an object)?
7. Describe a method that may be an improvement based on your perceived drawbacks.


**Datasets** <br>
http://rdatasciencecases.org/Data/offline.final.trace.txt <br>
http://rdatasciencecases.org/Data/online.final.trace.txt


### Data Description
* signal strength measurements are recorded between wireless hand-held device and routers
* floor plan of test environment measures [15 meters x 36 meters]; measurments were only taken in the hallway

* offline synonyomous with training data, but more accurately described as reference data of signal strength measurements at known locations
    
    * signal strength measured using a hand-held device at 166 different locations spaced 1 meter apart **(gray dots)**
    routers
    * router locations are located at 6 WiFi access points **(black squares)**
    
    * reference locations (x,y) coordinates of the hand-held device were each recorded in 45 degree increments for 8 different orientations
    * 110 signal strength measurements recorded to each of the 6 routers for all location-oritentation combinations
    
* online synonymous with test data; use the model built on the offline data to predict location of new unknown devices based on detected signals

    * 60 location-orientation pairs were randomly selected on the floor (black dots)
        *   110 signals measured from these sampled positions to each router 
        
<br>
<center><img src = "http://www.rdatasciencecases.org/GeoLoc/images/building.png"></center>
<br>

Training a model on data based on reference locations provides calibrated signal strength measurements at those locations within the environment, which will be used to predict the location of hand-held devices at uncalibrated positions.

**Note: Not all signal strengths were measured at every location. Measurements from other hand-held devices in the vicinity of the experimental unit also appear.**

#### Feature Description

* t: timestamp in milliseconds since midnight, January 1, 1970 UTC
* id: MAC address of the scanning device
* pos: the physical coordinate of the scanning device
* degree: oritentaion of the user carrying the scanning device in degrees
* MAC address: address of device with corresponding values for signal strength in (dBm), channel frequency (1-3), and mode (1)
    * a unique identifier that allows network cards in computer, access points, or other wireless equipment identified on the network

### Pre-Process Data

In [1]:
#from functools import partial
import rpy2.ipython
#rom rpy2.ipython import html
#html.html_rdataframe=partial(html.html_rdataframe, table_class="docutils")
import rpy2.robjects as robject
from rpy2.robjects.packages import importr
#from rpy2.robjects.packages import importr
#from rpy2.robjects import pandas2ri
#from rpy2.robjects.conversion import localconverter

In [2]:
%load_ext rpy2.ipython

In [123]:
%%time
%%R 
# DO NOT NEED TO RUN THIS CELL IF FILE ALREADY EXITS IN WORKING DIRECTORY

library(tidyverse)
library(magrittr)
# Parser Function that returns matricies of measured signal strengths
processLine = 
  function(x) {
  tokens = strsplit(x, "[;=,]")[[1]]
  
  if (length(tokens) == 10) 
    return(NULL)
  
  tmp = matrix(tokens[ - (1:10) ], , 4, byrow = TRUE)
  cbind(matrix(tokens[c(2, 4, 6:8, 10)], nrow(tmp), 6, 
               byrow = TRUE), tmp)
  }

roundOrientation = 
  function(angles) {
  refs = seq(0, by = 45, length = 9)
  q = sapply(angles, function(o) which.min(abs(o - refs)))
  c(refs[1:8], 0)[q]
  }

processData = 
  function(filename)
  { # read in data from file
    txt = readLines(filename)
    # subset original offline text data to remove all lines with comments
    lines = txt[ substr(txt, 1, 1) != "#" ]
    # apply parser function to every line in the text file
    tmp = lapply(lines, processLine)
    # cast parsed data into data frame 
    offline = as.data.frame(do.call("rbind", tmp), 
                            stringsAsFactors= FALSE) 
    # assign variable names to data features
    names(offline) = c("time", "scanMac", 
                       "posX", "posY", "posZ", "orientation", 
                       "mac", "signal", "channel", "type")
    
    # Remove all adhoc type measurements from data frame
    # Analysis will only include signal strengths measured to fixed access points
    offline = offline[ offline$type == "3", ]
    
    # Drop scanMac, posZ, channel, and type (contain no useful info)
    # only 1 value for scanMac, the MAC for the hand-held device
    # posZ = 0 for all measurements since they were all taken on the same floor
    dropVars = c("scanMac", "posZ", "channel", "type")
    offline = offline[ , !( names(offline) %in% dropVars ) ]
    
    # There are 12 unqiue MAC addresses and 8 channels
    # keep records from top 7 devices, since there is a large drop in number of signals after (>100,000)
    # channels removed b/c one-to-one correspondence between MAC addresses and channels for remaining 7 devices
    keepMacs = names(sort(table(offline$mac), decreasing = TRUE))[1:7]
    offline = offline[ offline$mac %in% keepMacs, ]
    
    # # convert attributes to appropriate data types (numeric)
    numVars = c("time", "posX", "posY", "orientation", "signal")
    offline[ numVars ] = lapply(offline[ numVars ], as.numeric)

    # Format time to POSIXt - change scale from milliseconds to seconds
    offline$rawTime = offline$time
    offline$time = offline$time/1000
    class(offline$time) = c("POSIXt", "POSIXct")
    
    # Round orientations to nearest 45 degree increment
    offline$orientation = roundOrientation(offline$orientation)
    
    # Cast data frame to widen it by putting the mac values and their associated signals in the columns
    # offline <- offline %>% pivot_wider(names_from = mac,values_from = signal, values_fn = list(signal=mean))

    return(offline)
  }

offline = processData("http://rdatasciencecases.org/Data/offline.final.trace.txt")
# Save Processed data into csv that can be read into python as a pandas dataframe 
write.csv(offline, file = 'offlineOutput.csv')

Wall time: 3min 39s


In [3]:
# Read in processed data as pandas dataframe
import pandas as pd
import numpy as np
offline = pd.read_csv("offlineOutput.csv")
# Remove index column
offline = offline.drop(columns=["Unnamed: 0"])

In [4]:
# cast time column as date-time data type
offline.time = pd.to_datetime(offline.time)
# create categorical factor levels for each mac address of fixed access points
offline['mac'] = offline['mac'].astype("category")

In [208]:
pd.pivot_table(offline, columns = "mac", values = "signal", aggfunc = np.mean)

mac,00:0f:a3:39:dd:cd,00:0f:a3:39:e1:c0,00:14:bf:3b:c7:c6,00:14:bf:b1:97:81,00:14:bf:b1:97:8a,00:14:bf:b1:97:8d,00:14:bf:b1:97:90
signal,-70.462247,-53.701821,-60.758609,-56.398183,-56.650013,-53.740144,-67.034967


In [217]:
test = offline.groupby(['time','posX','posY','orientation','rawTime'])

#for key, item in grouped_df:
#    print(grouped_df.get_group(key), "\n\n")

In [206]:
#offline.groupby(['time','posX','posY','orientation','rawTime']).mean()
len(offline.groupby(['time','posX','posY','orientation','rawTime']))

146074

In [221]:
offline.pivot_table(index = test.indices, columns = 'mac', values = 'signal', aggfunc = np.mean)

KeyError: (numpy.datetime64('2006-02-10T23:31:58.000000000'), 0, 0, 0, 1139643118358)

In [220]:
type(test)

pandas.core.groupby.generic.DataFrameGroupBy

In [167]:
110*60*7

46200

In [76]:
len(pd.unique(offline['time']))

37841